### Calculate error rate of prediction of different model

In [152]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [153]:
veg2 = ['香蕉','洋香瓜-網狀紅肉','蓮霧-紅蓮霧','鳳梨-金鑽鳳梨','南瓜-木瓜形',
        '大蒜-蒜仁','青蔥-日蔥','小白菜-土白菜','芽菜類-黃豆牙','木瓜-網室紅肉']

models = ['LSTM1','LSTM2','ARIMA','Prophet']

In [154]:
# Change vegetable class id
idx = 0
###########################
dataset = pd.read_csv('./All_Predict/All_Predict_'+veg2[idx]+'_'+'Price.csv')

In [155]:
dataset

,date,Price_0,LSTM1_0,LSTM2_0,ARIMA_0,Prophet_0
0,2019-01-01,26.4,26.483630,31.544243,26.961843,35.338524
1,2019-01-02,26.3,26.598310,29.218332,26.598490,35.624145
2,2019-01-03,27.3,26.788643,30.040482,26.491212,35.747380
3,2019-01-04,26.9,28.086271,36.360226,27.563889,35.371132
4,2019-01-05,29.4,28.341724,40.025210,27.134836,35.882735
...,...,...,...,...,...,...
293,2019-12-25,34.8,33.849620,33.317180,36.140011,35.580035
294,2019-12-26,34.0,34.254734,32.954510,35.604262,35.749286
295,2019-12-27,33.2,33.588028,32.298940,34.746996,35.440917
296,2019-12-28,32.6,33.192840,32.540900,33.889647,36.012947


In [156]:
for i in range(len(dataset)):
    if pd.isnull(dataset['Prophet_'+str(idx)][i]):
        print(dataset['date'][i],dataset['Prophet_'+str(idx)][i])
        dataset['Prophet_'+str(idx)][i] = dataset['Prophet_'+str(idx)][i-1]

2019-02-04 nan
2019-06-16 nan
2019-09-22 nan


c:\users\user\anaconda3\envs\lstm-shap\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [157]:
def normalize(target,y_max,y_min,y_mean):
    target_denorm = target.apply(lambda y: (y - y_mean)/(y_max - y_min))
    return  target_denorm

In [158]:
def de_normalize(test,y_max,y_min,y_mean):
    test_denorm = test.apply(lambda y: y * (y_max - y_min) + y_mean)
    return test_denorm

In [159]:
now_target = 'Price_'+str(idx)
y_max = np.max(dataset[now_target])
y_min = np.min(dataset[now_target])
y_mean = np.mean(dataset[now_target])

In [160]:
price_norm = normalize(dataset[now_target],y_max,y_min,y_mean)
LSTM1_norm = normalize(dataset['LSTM1_'+str(idx)],y_max,y_min,y_mean)
LSTM2_norm = normalize(dataset['LSTM2_'+str(idx)],y_max,y_min,y_mean)
ARIMA_norm = normalize(dataset['ARIMA_'+str(idx)],y_max,y_min,y_mean)
Prophet_norm = normalize(dataset['Prophet_'+str(idx)],y_max,y_min,y_mean)

In [161]:
print('LSTM1: ',veg2[idx],mean_squared_error(price_norm, LSTM1_norm))
print('LSTM2: ',veg2[idx],mean_squared_error(price_norm, LSTM2_norm))
print('ARIMA: ',veg2[idx],mean_squared_error(price_norm, ARIMA_norm))
print('Prophet: ',veg2[idx],mean_squared_error(price_norm, Prophet_norm))

LSTM1:  香蕉 0.010911565406999151
LSTM2:  香蕉 0.6134043527773795
ARIMA:  香蕉 0.005280327102344935
Prophet:  香蕉 0.10826459419474709


In [162]:
pd.concat([price_norm, LSTM1_norm, LSTM2_norm, ARIMA_norm, Prophet_norm], axis=1)

,Price_0,LSTM1_0,LSTM2_0,ARIMA_0,Prophet_0
0,-0.039677,-0.035993,0.186941,-0.014926,0.354090
1,-0.044082,-0.030941,0.084478,-0.030933,0.366673
2,-0.000030,-0.022556,0.120697,-0.035659,0.372102
3,-0.017651,0.034608,0.399099,0.011595,0.355527
4,0.092481,0.045861,0.560552,-0.007306,0.378064
...,...,...,...,...,...
293,0.330367,0.288500,0.265044,0.389398,0.364730
294,0.295125,0.306346,0.249068,0.365797,0.372186
295,0.259882,0.276976,0.220188,0.328032,0.358601
296,0.233451,0.259567,0.230847,0.290263,0.383801


In [163]:
price_denorm = de_normalize(price_norm,y_max,y_min,y_mean)
LSTM1_denorm = de_normalize(LSTM1_norm,y_max,y_min,y_mean)
LSTM2_denorm = de_normalize(LSTM2_norm,y_max,y_min,y_mean)
ARIMA_denorm = de_normalize(ARIMA_norm,y_max,y_min,y_mean)
Prophet_denorm = de_normalize(Prophet_norm,y_max,y_min,y_mean)

In [164]:
pd.concat([price_denorm, LSTM1_denorm, LSTM2_denorm, ARIMA_denorm, Prophet_denorm], axis=1)

,Price_0,LSTM1_0,LSTM2_0,ARIMA_0,Prophet_0
0,26.4,26.483630,31.544243,26.961843,35.338524
1,26.3,26.598310,29.218332,26.598490,35.624145
2,27.3,26.788643,30.040482,26.491212,35.747380
3,26.9,28.086271,36.360226,27.563889,35.371132
4,29.4,28.341724,40.025210,27.134836,35.882735
...,...,...,...,...,...
293,34.8,33.849620,33.317180,36.140011,35.580035
294,34.0,34.254734,32.954510,35.604262,35.749286
295,33.2,33.588028,32.298940,34.746996,35.440917
296,32.6,33.192840,32.540900,33.889647,36.012947


In [141]:
df = pd.read_csv('香蕉_Price.csv')

In [142]:
df

,DateTime,Original,Predict,Loss,MSE,Mean
0,2019-01-01,26.9,29.192320,0.041243,0.041243,12.162018
1,2019-01-02,26.4,31.544243,0.041243,0.041243,12.162018
2,2019-01-03,26.3,29.218332,0.041243,0.041243,12.162018
3,2019-01-04,27.3,30.040482,0.041243,0.041243,12.162018
4,2019-01-05,26.9,36.360226,0.041243,0.041243,12.162018
...,...,...,...,...,...,...
294,2019-12-26,34.8,33.317180,0.041243,0.041243,12.162018
295,2019-12-27,34.0,32.954510,0.041243,0.041243,12.162018
296,2019-12-28,33.2,32.298940,0.041243,0.041243,12.162018
297,2019-12-29,32.6,32.540900,0.041243,0.041243,12.162018


In [145]:
test_max = np.max(df['Original'])
test_min = np.min(df['Original'])
test_mean = np.mean(df['Original'])

In [147]:
original_norm = normalize(df['Original'],test_max,test_min,test_mean)
predict_norm = normalize(df['Predict'],test_max,test_min,test_mean)

In [148]:
mean_squared_error(original_norm, predict_norm)

0.6113869389513095

In [149]:
pd.concat([original_norm, predict_norm], axis=1)

,Original,Predict
0,-0.017592,0.083392
1,-0.039618,0.187001
2,-0.044023,0.084537
3,0.000029,0.120756
4,-0.017592,0.399158
...,...,...
294,0.330426,0.265103
295,0.295184,0.249127
296,0.259941,0.220247
297,0.233510,0.230906


In [150]:
original_denorm = de_normalize(original_norm,test_max,test_min,test_mean)
predict_denorm = de_normalize(predict_norm,test_max,test_min,test_mean)

In [151]:
pd.concat([original_denorm, predict_denorm], axis=1)

,Original,Predict
0,26.9,29.192320
1,26.4,31.544243
2,26.3,29.218332
3,27.3,30.040482
4,26.9,36.360226
...,...,...
294,34.8,33.317180
295,34.0,32.954510
296,33.2,32.298940
297,32.6,32.540900
